# Master TIDE - Conférences Python 2021

Francis Wolinski

&copy; 2021 Yotta Conseil


# 5. Opérations de pivot

Cette séance utilise les données accessibles sur le site de la Sécurité Sociale US.

Prénom, genre et nombre de naissances des enfants nés aux USA depuis 1880.

Seuls les prénoms avec au moins 5 naissances dans une année sont présents.

In [ ]:
# import des modules usuels
import numpy as np
import pandas as pd
from pandas import CategoricalDtype
import matplotlib.pyplot as plt
import seaborn as sns

# options d'affichage
pd.set_option("display.max_rows", 16)
plt.style.use('seaborn-darkgrid')

In [ ]:
# names
# https://www.ssa.gov/oact/babynames/names.zip

from os.path import exists

if exists("names.pkl"):   
    df = pd.read_pickle("names.pkl") # restauration à partir du format pickle

else:
    import glob # Python module qui implémente la fonction glob()

    files =  glob.glob('names/*.txt')  # lecture des fichiers .txt
    files.sort()
    dfs = []
    for filename in files:
        year = int(filename[-8:-4]) # extraction de l'année
        csv = pd.read_csv(filename, names=['name', 'gender', 'births']) # chargement d'un fichier unitaire
        csv['year'] = year # ajout de la colonne année
        dfs.append(csv)
        
    df = pd.concat(dfs, ignore_index=True) # concaténation de tous les DataFrames

    df = df[['year', 'name', 'gender', 'births']] # ordre des colonnes
    df.to_pickle("names.pkl") # sauvegarde au format pickle
    
df.shape

In [ ]:
df

## 5.1 Fonction `crosstab()`

La fonction `crosstab()` calcule les modalités croisées de deux facteurs (2 objets de type `Series` partageant le même index. Par ex, 2 colonnes d'un même `DataFrame`).

L'option `margine=True` calcule le total par ligne et par colonne. L'option `normalize=...` normalise les résultats en pourcentages compris entre 0 et 1 (`True` : toutes les valeurs, `index` : par ligne, `columns` par colonne).

Le résultat est un objet de type `DataFrame` : l'index comprend les différentes modalités de la première `Series` et les colonnes les différentes modalités de la seconde `Series`.

In [ ]:
# ajout de la longueur des prénoms
df['length'] = df['name'].str.len()
df

In [ ]:
# crosstable simple
pd.crosstab(df['length'], df['gender'])

## 5.2 La méthode `pivot_table()`

La méthode `pivot_table()` construit un tableau synthétique de valeurs agrégées et ventilées selon les différentes valeurs d'une ou plusieurs colonnes.

Elle retourne un nouveau `DataFrame` en fonction des paramètres fournis.

- values : colonnes du `DataFrame` initial dont les valeurs sont agrégées
- index : colonnes du `DataFrame` initial dont les valeurs sont utilisées en index
- columns : colonnes du `DataFrame` initial dont les valeurs sont utilisées en nom de colonnes
- aggfunc : fonction d'agrégation des valeurs, par défaut `numpy.mean` (calcul de la moyenne des valeurs), : mean, min, max, sum, count, nunique, median, fonction ou lambda.

In [ ]:
# pivot table naissances par année et genres
var = df.pivot_table(values='births',
            index='year',
            columns='gender',
            aggfunc='sum')
var

In [ ]:
# vérification
df.loc[(df['year']==1880)&(df['gender']=='F'), 'births'].sum()

In [ ]:
# graphique
var.plot(title='Nombre de naissances par année et par genre'); # pour changer les couleurs par exemple color=['m', 'c']

In [ ]:
# sans l'option columns
var = df.pivot_table(values='births',
            index='year',
            aggfunc='sum')
var

<div class="alert alert-success">
<b>Exercice 1</b>
<ul>
    <li>Pivot tables avec le premier ou le dernier nom par ordre alphabétique par année et par genre.</li>
</ul>

<div class="alert alert-success">
<b>Exercice 2</b>
<ul>
    <li>Pivot table avec la diversité des prénoms (nombre de prénoms différents) par année et par genre.</li>
    <li>Vérifier pour les femmes en 1880.</li>
    <li>Calcul de la différence de diversité des prénoms entre les hommes et les femmes par année</li>
    <li>Calcul du maximum de cette différence. Pour quelle année ?</li>
    <li>Faire un graphique.</li>
</ul>

## 6. Gestion des valeurs manquantes et des doublons

### 6.1 Valeurs manquantes

Le module pandas possède des fonctions pour gérer les valeurs manquantes.

In [ ]:
# pivot table of a subset of df with name equals to mary
var = df.loc[(df['name']=='Mary')]
tab = var.pivot_table(values='births',
                        index='year',
                        columns='gender',
                        aggfunc='sum')
tab

On constate que la valeur `NaN` est affichée dans certaines lignes.

Il s'agit de la valeur `np.nan`, `NaN` signifiant "Not a Number".

`NaN` est objet spécial introduit par `NumPy` pour représenter l'absence de valeur.

Il est à noter que la fonction `crosstab()` ne produit pas de `NaN` mais met des 0.

In [ ]:
# crosstab année x genre pour le prénom Mary
pd.crosstab(var['year'], var['gender'])

In [ ]:
# accès à la valeur
tab.iloc[-2, -1]

In [ ]:
# l'objet NaN de NumPy
np.nan

In [ ]:
# NaN est un nombre flottant
type(np.nan)

In [ ]:
# arithmétique de NaN : NaN est un élément absorbant
np.nan + 1  # avec n'importe quelle opération mathématique

In [ ]:
# toute opération avec NaN renvoie Nan
1 + np.nan

In [ ]:
# fonctions
np.sqrt(np.nan)  # avec n'importe quelle fonction mathématique

In [ ]:
np.nan == np.nan

La librairie pandas propose toute une panoplie d'outils pour gérer l'absence de valeur, phénomène fréquent lorsque l'on fait de l'analyse de données.

In [ ]:
tab == np.nan

In [ ]:
# test sur la présence de Nan
tab.isnull()

In [ ]:
# test sur l'absence de Nan
tab.notnull()

In [ ]:
# lignes ayant au moins un NaN
tab.loc[tab.isnull().any(axis=1)]

In [ ]:
# suppression des lignes comportant un Nan
tab.dropna()

In [ ]:
# suppression des colonnes comportant un Nan
tab.dropna(axis=1)

In [ ]:
# remplacement des NaN par une valeur fixe
tab.fillna(0)

Il existe d'autres méthodes de remplissage (ffill, bfill), voir http://pandas.pydata.org/pandas-docs/stable/missing_data.html

Il faut aussi étudier les options de la fonction `read_csv()` :

- na_values
- keep_default_na
- na_filter

In [ ]:
# read_csv()

pd.read_csv?

### 6.2 Doublons

Il est également possible de supprimer les éventuelles lignes dupliquées. La méthode `duplicated()` détermine si les lignes sont dupliquées ou non et la méthode `drop_duplicates()` supprime les doublons (seule la première ligne est conservée).

In [ ]:
# décompte des lignes dupliquées
df[['year', 'name']].duplicated().value_counts()

In [ ]:
# supression des lignes dupliquées
var = df[['year', 'name']].drop_duplicates()
var.shape

## Exercices complémentaires

<div class="alert alert-success">
<b>Exercice 3</b>
<p>Nous allons tracer l'évolution de la diversité des prénoms se terminant par une lettre donnée, et ce, pour les 7 lettres ayant le plus de diversité en 2018.</p>
    <ul>
    <li>Ajouter une colone au DataFrame avec la dernière lettre de chaque prénom en majuscule</li>
    <li>Calculer un tableau croisé avec les années et les lettres terminales donnant le nombre de prénoms</li>
    <li>Sélectionner les 7 plus grandes valeurs de la ligne correspondant à la dernière année</li>
    <li>Tracer le tableau croisé obtenu limité aux 7 lettres correspondant aux 7 plus grandes valeurs trouvées</li>
    </ul>
</div>

<div class="alert alert-success">
<b>Exercice 4</b>
<p>Etude des prénoms qui ont changé de genre 1/2</p>
    <ul>
    <li>Faire un value counts des prénoms. Quelle interprétation ?</li>
    <li>Calculer la liste des prénoms qui apparaissent au moins 10000 fois.</li>
    <li>Sélectionner ces prénoms dans le DataFrame.</li>
    <li>Pour chaque prénom calculer le nombre de naissances par genre et le ratio "F/(F + M)".</li>
    <li>Afficher un histogramme du ratio trouvé.</li>
    </ul>
</div>

<div class="alert alert-success">
<b>Exercice 5</b>
<p>Etude des prénoms qui ont changé de genre 2/2</p>
    <ul>
    <li>Sélectionner le prénom "Leslie".</li>
    <li>Calculer les ratios "F/(F + M)" et "M/(F + M)" avec la méthode <code>div()</code>.</li>
    <li>Afficher un graphe avec l'évolution de ces ratios dans le temps.</li>
    <li>Essayer avec d'autres prénoms : Donnie, Jean, Angel, Kim, Sydney.</li>
    </ul>
</div>